## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_DataBase.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Geraldton,AU,-28.7667,114.6000,66.61,82,40,18.41,scattered clouds
1,1,Kodiak,US,57.7900,-152.4072,17.11,49,1,26.46,clear sky
2,2,New Norfolk,AU,-42.7826,147.0587,51.93,72,37,1.99,scattered clouds
3,3,Los Llanos De Aridane,ES,28.6585,-17.9182,64.13,63,40,3.00,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,51.46,62,75,14.97,broken clouds


In [3]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
print("What is your minimum temperature preference?")
min_temp = float(input("min_temp: "))

print("What is your maximum temperature preference?")
max_temp = float(input("max_temp: "))


What is your minimum temperature preference?
min_temp: 50
What is your maximum temperature preference?
max_temp: 75


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Geraldton,AU,-28.7667,114.6000,66.61,82,40,18.41,scattered clouds
2,2,New Norfolk,AU,-42.7826,147.0587,51.93,72,37,1.99,scattered clouds
3,3,Los Llanos De Aridane,ES,28.6585,-17.9182,64.13,63,40,3.00,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,51.46,62,75,14.97,broken clouds
6,6,Bluff,NZ,-46.6000,168.3333,57.79,95,100,3.24,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(len(cities_preferred_df))
cities_preferred_df.count()

176


Unnamed: 0             176
City                   176
Country                172
Lat                    176
Lng                    176
Max Temp               176
Humidity               176
Cloudiness             176
Wind Speed             176
Current Description    176
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#clean_df = cities_preferred_df.dropna(axis="index", how="any")
#print(len(clean_df))
#clean_df.head()
clean_df = cities_preferred_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Geraldton,AU,66.61,scattered clouds,-28.7667,114.6000,
2,New Norfolk,AU,51.93,scattered clouds,-42.7826,147.0587,
3,Los Llanos De Aridane,ES,64.13,scattered clouds,28.6585,-17.9182,
4,Ushuaia,AR,51.46,broken clouds,-54.8000,-68.3000,
6,Bluff,NZ,57.79,overcast clouds,-46.6000,168.3333,
7,Busselton,AU,53.04,clear sky,-33.6500,115.3333,
11,Albany,US,50.27,overcast clouds,42.6001,-73.9662,
17,Barcelona,ES,53.85,few clouds,41.3888,2.1590,
19,Guerrero Negro,MX,72.14,clear sky,27.9769,-114.0611,
22,Mildura,AU,53.08,few clouds,-34.2000,142.1500,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Geraldton,AU,66.61,scattered clouds,-28.7667,114.6000,Broadwater Mariner Resort
2,New Norfolk,AU,51.93,scattered clouds,-42.7826,147.0587,The Shingles Riverside Cottages
3,Los Llanos De Aridane,ES,64.13,scattered clouds,28.6585,-17.9182,Valle Aridane
4,Ushuaia,AR,51.46,broken clouds,-54.8000,-68.3000,Albatros Hotel
6,Bluff,NZ,57.79,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park


In [11]:
# 8a. Create the output File (CSV)
output_data_filev= "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("Vacation_Search.csv", index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))